In [1]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Regression

## 数据读取与处理

In [2]:
from sklearn.datasets import load_boston

data = load_boston()
X = data.data
Y = data.target

scaler=MinMaxScaler()
X_sca=scaler.fit_transform(X)

X_train,X_val,Y_train,Y_val=train_test_split(X_sca,Y,test_size=0.15)

print(X_train.shape, Y_train.shape,X_val.shape,Y_val.shape)

(430, 13) (430,) (76, 13) (76,)


## 网络结构设计

In [3]:
# 神经网络编程有一点与SKlearn不一样，就是需要提前设计好网络的结构，每一层的参数都跟其输入有关
# 输入层单元数为13(数据特征)，设计隐藏层单元数为5，输出单元数为1(回归任务)

unit_I = X_train.shape[1]
unit_1 = 5
unit_O = 1

## 搭建网络

In [8]:
# 为了便于添加层数，将其封装起来
def add_layer(inputs,units_I,units_O,act_func=tf.nn.relu,layer_name=None):
    w = tf.Variable(tf.random_normal(shape=[units_I, units_O]))
    b = tf.Variable(tf.zeros([1, units_O]))
    try:
        out = act_func(tf.matmul(inputs, w)+b)
    except Exception as e:
        print(e)
    return out

In [9]:
# 输入必须是可由用户指定的，所以设为placeholder
X = tf.placeholder(tf.float32, shape=[None, unit_I])
Y = tf.placeholder(tf.float32, shape=(None))    # 目标值为一维向量

# 网络结构子图
with tf.name_scope('DNN'):
    a1 = add_layer(X, unit_I, unit_1)
    Y_pred = add_layer(a1, unit_1, unit_O)

# 目标函数子图
with tf.name_scope('Loss'):
    MSE = tf.reduce_mean(tf.squared_difference(Y, Y_pred))

# optimization子图
lr = 0.01
with tf.name_scope('Train'):
    opt = tf.train.GradientDescentOptimizer(lr).minimize(MSE)    # GD优化器

init = tf.global_variables_initializer()    # 所有变量初始化

## 训练网络

In [ ]:
# 计算图已经构建好，开启一个tf会话，需要计算哪个值就run哪个变量即可
with tf.Session() as sess:
    sess.run(init)
    train_iter = 1000

    for i in range(train_iter):
        sess.run(opt, feed_dict={X: X_train, Y: Y_train})
        if i % 100 == 0:
            mse_train = sess.run(MSE, feed_dict={X: X_train, Y: Y_train})
            print('train_iter: {}\tMSE:{}'.format(i, mse_train))

    mse_val = sess.run(MSE, feed_dict={X: X_val, Y: Y_val})
    
    # 另一种计算方式，因为tf计算出来的pred是一个二维数组，需要squeeze一下
#     pred = sess.run(Y_pred, feed_dict={X: X_val})
#     mse_val=np.sum(np.square(np.squeeze(pred)-Y_val))/len(Y_val)
    
    print('test_MSE: {}'.format(mse_val))

# Binary classification

In [6]:
# from sklearn.datasets import load_breast_cancer

# data = load_breast_cancer()
# X = data.data
# Y = data.target

# scaler=MinMaxScaler()
# X_sca=scaler.fit_transform(X)

# X_train,X_val,Y_train,Y_val=train_test_split(X_sca,Y,test_size=0.15)

# print(X_train.shape, Y_train.shape,X_val.shape,Y_val.shape)

# Multi classification